In [187]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import types
import time
import re
import queue
import string

In [188]:
#Get Seed URLs and URLs from the seed URL
def getQueue(url):
    q = queue.Queue()
    r = requests.get("http://" + url)
    data = r.text
    soup = BeautifulSoup(data)
    for link in soup.find_all('a'):
        link = link.get('href')
        if link!=None:
            q.put(link)
            print (link)
    return q
q=getQueue('en.wikipedia.org/wiki/Web_mining')

#mw-head
#p-search
/wiki/Wikipedia:Cleanup
/wiki/Wikipedia:Manual_of_Style
/wiki/Template:Cleanup/doc
//en.wikipedia.org/w/index.php?title=Web_mining&action=edit
/wiki/Help:Maintenance_template_removal
/wiki/Data_mining
/wiki/World_Wide_Web
#Web_usage_mining
#Pros
#Cons
#Web_structure_mining
#Web_content_mining
#Web_mining_in_foreign_languages
#Chinese
#See_also
#References
#Resources
#External_links
#Books
#Bibliographic_references
/w/index.php?title=Web_mining&action=edit&section=1
/wiki/Data_mining
/w/index.php?title=Usage_pattern&action=edit&redlink=1
/wiki/Web-based_application
/w/index.php?title=Browsing_behavior&action=edit&redlink=1
/wiki/Web_server
/wiki/E-commerce
/wiki/Semantic
/wiki/Hyponymy
/w/index.php?title=Web_mining&action=edit&section=2
/wiki/Personalized_marketing
/wiki/Fight_against_terrorism
/wiki/Customer_relationship
/wiki/Profitability
/wiki/Target_pricing
/wiki/User_profiling
/w/index.php?title=Web_mining&action=edit&section=3
/wiki/Personal_data
/wiki/Internet

C:\Users\amuth\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\amuth\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [189]:
#Determine if page is relevant (before downloading)
def classRel(url):
    #html = urlopen(url).read()
    #soup = BeautifulSoup(html)
    #text = soup.text()
    return True

In [190]:
#Make sure the url is valid before crawling (fix!!0)
def makeVal(url):
    if url[1]=='w':
        url = "http://en.wikipedia.org" + url
        return url
    else:
        return None
#while not q.empty():
    #makeVal(q.get())

In [191]:
#Create a valid file name for the downloaded page
def getName(url):
    name = url.split('/')[-1]
    if '.' in name:
        name = name.split('.')[0]
    return name

In [192]:
#Download pages from the urls in the queue
def getPage(url):
    url = makeVal(url)
    #handle HTTPErrors here
    html = urlopen(url).read()
    path='Crawled_Pages/'
    name = getName(url)
    if classRel(url):
        file = open(path+name+'.html','w')
        file.write(str(html))
        file.close()  
while not q.empty():
    url = q.get()
    if makeVal(url) != None:
        getPage(url)
    else:
        continue

HTTPError: HTTP Error 404: Not Found